<a href="https://colab.research.google.com/github/TheBluthCompany/FI424-Python-DCF/blob/main/Evercore_DCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !!! Check out my other work! (Artificial Intelligence, Machine Learning, Neural Networks)
#Let's invite all our friends to the houseparty!
import numpy as np
import pandas as pd 
!pip install openpyxl
!pip install yfinance
import os 

#Import files downloaded from FRED 
from google.colab import files
files.upload()

#Set row names instead of integers using inplace = True
balance_sheet = pd.read_excel('Evercore Balance Sheet.xlsx')
balance_sheet.set_index(
    'Consolidated Statements of Financial Condition - USD ($) $ in Thousands', inplace = True)

income_statement = pd.read_excel('Evercore Income Statement.xlsx')
income_statement.set_index(
    'Consolidated Statements of Operations - USD ($) shares in Thousands, $ in Thousands', inplace = True)

cash_flow_statement = pd.read_excel('Evercore Cash Flow Statement.xlsx')
cash_flow_statement.set_index(
    'Consolidated Statements of Cash Flows - USD ($) $ in Thousands', inplace = True)

securities_outstanding = pd.read_excel('Evercore Securities Outstanding.xlsx')
securities_outstanding.set_index(
    'Consolidated Statements of Financial Condition (Parenthetical) - USD ($) $ in Thousands', inplace = True)


# - - - - - - - - - - - - - - - Create a Linear Regression Model to Forecast FCF - - - - - - - - - - - - - - - 
from sklearn.linear_model import LinearRegression

x = np.arange(len(income_statement.loc['Net Revenues'].iloc[0])).reshape(-1, 1)


stock_chart = LinearRegression()
stock_chart.fit(x, income_statement.loc['Net Revenues'].iloc[0])

forecast_period = 5

forecasted_revenue = stock_chart.predict(np.arange(len(income_statement.loc['Net Revenues'].iloc[0]) + forecast_period).reshape(-1, 1))

cash_and_cash_equivalents = (balance_sheet.loc['Cash and Cash Equivalents'].iloc[0])

future_net_income = (forecasted_revenue) * (income_statement.loc['Net Income'].iloc[0])

future_free_cash_flow = cash_and_cash_equivalents + future_net_income

#Forecast the FCF
forecasted_cash_flows = []
for i in range(forecast_period):
    future_net_income = forecasted_revenue[i] * (income_statement.loc['Net Income'].iloc[0])
    future_free_cash_flow = (cash_and_cash_equivalents + future_net_income)
    forecasted_cash_flows.append(future_free_cash_flow)


#- - - - - - - - - - - - - - - Debt/Equity Values - - - - - - - - - - - - - - - 
import yfinance as yf

#Find the shares outstanding 
outstanding_shares = (
    securities_outstanding.loc['Common Stock, Shares, Outstanding'].iloc[0,0])

#Calculate the most recent stock price
stock = yf.Ticker('EVR')
current_stock_price = stock.info['regularMarketPrice']

# Calculate the market value of equity
market_value_of_equity = outstanding_shares * current_stock_price
market_value_of_equity = market_value_of_equity / 1000

#Calculate the Market Value of Debt
market_value_of_debt = (balance_sheet.loc['Total Liabilities'].iloc[0]
                        - balance_sheet.loc['Total Current Liabilities'].iloc[0])

market_value_of_debt = market_value_of_debt / 1000



#- - - - - - - - - - - - - - - Calculate the Cost of Equity using CAPM (Capital Asset Pricing Model)- - - - - - - - - - - - - - - - 

#Calculate the Risk Free Rate
yield_curve = yf.download(['^TNX'], 
                          start = '2015-01-01', 
                          end = '2021-01-01'
                          )
risk_free_rate = yield_curve['Adj Close'].mean()
risk_free_rate = risk_free_rate/100

#Calculate Beta
beta = stock.info['beta']
beta = beta / 100

#Calculate Average Market Return
sp500_curve = yf.download([
                                        'SPX'], 
                           start= '2015-01-01', 
                           end= '2021-01-01')

market = sp500_curve['Adj Close'].pct_change()
average_market_return = market.mean()

#Calculate the Cost of Equity 
cost_of_equity = risk_free_rate + beta * (average_market_return - risk_free_rate)
cost_of_equity = cost_of_equity / 1


#- - - - - - - - - - - - - - - Cost of Debt - - - - - - - - - - - - - - - 

#Calculate the Total Debt (simple)
average_total_debt = ((
                     balance_sheet.loc['Total Liabilities'].iloc[0] 
                     + balance_sheet.loc['Total Liabilities'].iloc[1])
                     - (balance_sheet.loc['Total Current Liabilities'].iloc[0]
                        + balance_sheet.loc['Total Current Liabilities'].iloc[1]
                     )) / 2

interest_rate = (
                     income_statement.loc['Interest Expense'].iloc[0]
                     / average_total_debt)

#Calculate the Tax Rate
tax_rate = (
    (income_statement.loc['Income Before Income Taxes'].iloc[0, 0]
     - income_statement.loc['Net Income'].iloc[0])
    / income_statement.loc['Income Before Income Taxes'].iloc[0, 0])

#Calculate the Cost of Debt
cost_of_debt = interest_rate * (1-tax_rate)


#- - - - - - - - - - - - - - - Calculate the Weighted Average Cost of Capital (WACC) - - - - - - - - - - - - - - - - - - - - - - - 
total_market_value = market_value_of_equity + market_value_of_debt
equity_weight = market_value_of_equity / total_market_value
debt_weight = market_value_of_debt / total_market_value

WACC = (
    (equity_weight * cost_of_equity) + ((cost_of_debt * debt_weight) * (1 - tax_rate)))

print("The Calculated WACC under conservative assumptions, for %s, is %.2f" %(stock.info['shortName'], WACC*100))

#To be continued this semester. 
yf.Ticker('EVR').info

